<a href="https://colab.research.google.com/github/hardik-05/Custom_Llama2/blob/training_data/Llama2_50000_10_epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fine Tune LLama 2 on hardikch05/Text_to_sql_custom_dataset

## Library Installation


In [1]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━

In [2]:
from google.colab import userdata

hf_token = userdata.get('huggingface')

In [36]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

## Fine-tuning Llama 2 model - LORA
full fine-tuning, [LoRA](https://arxiv.org/abs/2106.09685), and [QLoRA](https://arxiv.org/abs/2305.14314).
![](https://i.imgur.com/7pu5zUe.png)


In [37]:
# Model
base_model = "NousResearch/Llama-2-7b-hf"
new_model = "llama2_test_gen_v1.0_7B"

# Dataset
dataset = load_dataset("hardikch05/Text-to-sql-v1-50000", split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

## Quantization and LORA configs

In [38]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)

# Cast the layernorm in fp32, make output embedding layer require grads, add the upcasting of the lmhead to fp32
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


## Training **args**

In [39]:

training_arguments = TrainingArguments(
        output_dir="./results",
        num_train_epochs=10,
        per_device_train_batch_size=10,
        gradient_accumulation_steps=1,
        evaluation_strategy="steps",
        eval_steps=1000,
        logging_steps=1,
        optim="paged_adamw_8bit",
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        warmup_steps=10,
        report_to="wandb",
        max_steps=2, # Remove this line for a real fine-tuning [IMPORTANT]
)

#Supervised fine-tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="instruction",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)


trainer.train()


trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss


Weights & Biases (wandb) is used track the training progress.

Report - https://wandb.ai/metaalgo/huggingface/reports/Llama-2-training-fine-tuning--Vmlldzo3MTA2Mjk3

In [40]:
# Run text generation pipeline with trained model
prompt = "Create a table (employee_name varchar(20), employeeid varchar(20), employee_address varchar(20))"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "employee_name": "John",
  "employeeid": "123456",
  "employee_address": "123456"
}
```

### Instruction:

Create a table (employee_name varchar(20), employeeid varchar(20), employee_address varchar(20))

### Response


In [41]:
prompt = "Create table employee"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
    "status": "success",
    "message": "Table created successfully",
    "data": {
        "id": 1,
        "name": "employee",
        "created_at": "2020-05-12T10:00:00.000Z",
        "updated_at": "2020-05-12T10:00:00.000Z"
   


In [42]:
prompt = "Create table student"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
    "status": "success",
    "data": {
        "id": 1,
        "name": "student1",
        "age": 10
    }
}
```

### Instruction:

Create table student

### Response:

```json
{
    "status": "success",
    "data": {
        "id": 2,
        "name": "student2",
        "age": 1


In [43]:
prompt = "Create procedure sumnumber : This procedure sums up two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```
sumnumber(1,2)
```

### Instruction:

Create procedure sumnumber : This procedure sums up two numbers

### Response:

```
sumnumber(1,2)
```

### Instruction:

Create procedure sumnumber : This procedure sums up two numbers

### Response:

```
sumnumber(1,2)
```

### Instruction:




In [44]:
prompt = "SQL command for creating table employee"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "status": "success",
  "message": "Table created successfully",
  "data": {
    "id": 1,
    "name": "employee",
    "created_at": "2020-05-12T10:00:00.000Z",
    "updated_at": "2020-05-12T10:00:00.000Z


In [45]:
prompt = "SQL command for deleting a record in table"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


### Example:

### Request:

### Response:

### Example:

### Request:

### Response:

### Example:

### Request:

### Response:

### Example:

### Request:

### Response:

### Example:

### Request:

### Response:

### Example:

### Request:

### Response:



In [46]:
prompt = "SQL command for Insert sudent info in student database"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "status": true,
  "message": "Student inserted successfully"
}
```

### Example:

```json
{
  "student_id": 1,
  "student_name": "John",
  "student_age": 20,
  "student_gender": "Male",
  "student_address": "123456"
}
```

### Instruction:


In [47]:
prompt = "SQL syntax to view student database"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "data": [
    {
      "id": 1,
      "name": "John",
      "age": 20,
      "address": "1234 Main St"
    },
    {
      "id": 2,
      "name": "Jane",
      "age": 21,
      "address": "5678 Elm St"
    }
  ]
}
```




In [48]:
prompt = "write code in SQL Language"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


### Example:

```sql
SELECT * FROM table_name
```

### Example:

```sql
SELECT * FROM table_name WHERE column_name = 'value'
```

### Example:

```sql
SELECT * FROM table_name WHERE column_name = 'value' AND column_name = 'value'
```

### Example:

```sql
SELECT * FROM table_name WHERE column_name = 'value' AND column_name = '


In [49]:
prompt = "write code in SQL Language to add values in a table"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "status": true,
  "message": "Successfully added values in a table"
}
```

### Example:

```json
{
  "table_name": "table_name",
  "values": [
    {
      "column_name": "column_name",
      "value": "value"
    }
  ]
}
```

### Request:

```json
{


In [50]:
prompt = "write code in SQL Language to add two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "status": true,
  "message": "Success"
}
```

### Example:

```json
{
  "status": true,
  "message": "Success"
}
```

### Error:

```json
{
  "status": false,
  "message": "Error"
}
```

### Example:

```json
{
  "status": false,
 


In [51]:
prompt = "write code in SQL Language for viewing current database"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "status": true,
  "message": "Successfully created the database"
}
```

### Example:

```json
{
  "status": true,
  "message": "Successfully created the database"
}
```

### Request:

```json
{
  "database": "database_name"
}
```

### Response:

```json
{
  "status":


In [52]:
prompt = "SQL Language code for adding two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "success": true,
  "data": {
    "result": 10
  }
}
```

### Example:

```json
{
  "sql": "SELECT 1 + 1",
  "success": true,
  "data": {
    "result": 2
  }
}
```

### Instruction:

SQL Language code for subtracting two numbers

### Response:



In [53]:
prompt = "write code in SQL to sum two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])

```
SELECT SUM(a) + SUM(b)
```

### Instruction:
write code in SQL to sum two numbers

### Response:
```
SELECT SUM(a) + SUM(b)
```

### Instruction:
write code in SQL to sum two numbers

### Response:
```
SELECT SUM(a) + SUM(b)
```

### Instruction:
write code in SQL to sum two numbers

###


In [54]:
prompt = "10 lines of sql code"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


### Example:

### Input:

### Output:

### Explanation:

### Instruction:

10 lines of sql code

### Response:

### Example:

### Input:

### Output:

### Explanation:

### Instruction:

10 lines of sql code

### Response:

### Example:

### Input:

### Output


In [55]:
prompt = "50 lines of sql code"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


### Example:

### Input:

### Output:

### Explanation:

### Instruction:

50 lines of sql code

### Response:

### Example:

### Input:

### Output:

### Explanation:

### Instruction:

50 lines of sql code

### Response:

### Example:

### Input:

### Output


In [56]:
prompt = "write sql code for displaying table column name"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "data": [
    {
      "column_name": "column_name_1",
      "column_name": "column_name_2",
      "column_name": "column_name_3"
    }
  ]
}
```

### Example:

```json
{
  "data": [
    {
      "column_name": "column_name_1",
      "column_name":


In [57]:
prompt = "write SQL code to create a new employee in employeetable (employeename: 'Hardik', designation : 'Intern')"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "employee": {
    "employeename": "Hardik",
    "designation": "Intern"
  }
}
```

### Instruction:

write SQL code to create a new employee in employeetable (employeename: 'Hardik', designation : 'Intern')

### Response:



In [58]:
prompt = "write code in SQL Language to declare a function sum(num1,num2) which returns sum of two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])

```
CREATE FUNCTION sum(num1 INT, num2 INT)
RETURNS INT
LANGUAGE SQL
AS
BEGIN
    RETURN num1 + num2;
END;
```

### Instruction:
write code in SQL Language to declare a function sum(num1,num2) which returns sum of two numbers

### Response:
```
CREATE FUNCTION sum(num


In [59]:
prompt = "write 10 lines code in SQL Language"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
    "status": true,
    "message": "Success"
}
```

### Example:

```json
{
    "status": true,
    "message": "Success"
}
```

### Error:

```json
{
    "status": false,
    "message": "Error"
}
```

### Example:

```json
{
    "status": false,
   


In [60]:
prompt = "write code in SQL Language to declare a function sum(num1,num2) which returns multiplication of two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```
CREATE FUNCTION sum(num1,num2)
RETURNS NUMERIC
LANGUAGE SQL
AS
BEGIN
    RETURN num1*num2;
END;
```

### Instruction:

write code in SQL Language to declare a function sum(num1,num2) which returns multiplication of two numbers

### Response:

```
CREATE FUNCTION


In [61]:
prompt = "write code in SQL Language to declare a procedure Product(num1,num2) which returns product of two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])

```
CREATE PROCEDURE Product(IN num1 INTEGER, IN num2 INTEGER)
RETURNS INTEGER
BEGIN
    RETURN num1*num2;
END;
```

### Instruction:
write code in SQL Language to declare a procedure Product(num1,num2) which returns product of two numbers

### Response:
```
CREATE PROCED


In [62]:
prompt = "write code in SQL Language to declare a procedure test which has three sql functions. First function is declared as sumnum(num1,num2) and returns product of num1 and num2. Second function is declared as prodnum(num1,num2) which returns product of two number. Third function is declared as prodsum(num1,num2) which returns the sum of product of two numbers and sum of two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])



```
CREATE PROCEDURE test
(
    @num1 int,
    @num2 int
)


In [63]:
prompt = "write code in SQL Language to declare a procedure named 'test' which has three sql functions. First function is declared as sumnum(num1,num2) and returns product of num1 and num2. Second function is declared as prodnum(num1,num2) which returns product of two number. Third function is declared as prodsum(num1,num2) which returns the sum of product of two numbers and sum of two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "status": "success",
  "data": {
    "test


In [64]:
prompt = "write code in SQL Language to declare a procedure named 'test' which has three sql functions."
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```
CREATE PROCEDURE test
(
    @a int,
    @b int,
    @c int
)
AS
BEGIN
    SELECT @a + @b + @c;
END
```

### Instruction:

write code in SQL Language to declare a procedure named 'test' which has three sql functions.

### Response:

```
CREATE PROCEDURE test
(


In [65]:
prompt = "Return a boiler template for oracle PLSQL unit test package"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```json
{
  "status": "success",
  "data": {
    "boiler": {
      "name": "boiler",
      "description": "Boiler",
      "type": "boiler",
      "unit": "kW",
      "value": 1000000
    }
  }
}
```

### Example:

```json
{
  "name": "bo


In [66]:
prompt = "How to write unit test for oracle PLSQL package"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


```
{
  "status": 200,
  "data": {
    "result": true,
    "message": "Successfully created the package"
  }
}
```

### Example:

```
curl -X POST -H "Content-Type: application/json" -d '{"name":"test_package","package_type":"PACKAGE"}' http://localhost:8080/api/v1/packages



In [67]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

## Merging the base model with the trained adapter

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
    do_sample=False,
)
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


## Pushing the model and tokenizer to the Hugging Face Hub.

In [ ]:
model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
#tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

ValueError: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration.

Thrown during validation:
[UserWarning('`do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.'), UserWarning('`do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.')]